**scikit-learn数据集和基本功能：**

In [4]:
from sklearn.datasets import *
iris = load_iris()
print(iris.data.shape)

(150, 4)


In [ ]:
import matplotlib.pyplot as plt
digits = load_digits()
#著名的手写数字数据集。
plt.matshow(digits.images[0])
plt.show()

**sklearn主要可以完成分类任务、回归任务、聚类任务、降维任务、模型选择和数据预处理**

**无监督学习解决聚类问题：**

**常用的距离测算法**
- 欧氏距离
- 曼哈顿距离
- 马氏距离（数据的协方差距离）
- 夹角余弦<br>
`sklearn.cluster`库中有很多聚类算法函数。

**无监督学习解决降维问题**
- 数据可视化
- 精简数据

**k-means聚类算法**
- 以$k$为参数，将对象分为$k$个簇，使簇内有较高的相似度，而簇间的相似度较低。
- 随机选择$k$个点作为初始的聚类中心
- 对于剩下的点，将其归入最近的簇
- 对每个簇计算所有点的均值作为新的聚类中心

**应用1：了解1999年各个省份我国的消费情况。**

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
#mooc没有给数据文件...?
def loadData(filePath):
    fr = open(filePath, 'r')
    lines = fr.readlines()
    retData = []
    retCityName = []
    for line in lines:
        items = line.strip().split(',')
        retCityName.append(line[0])
        retData.append([float(items[i]) for i in range(1, len(items))])
    return retData, retCityName

if __name__ == '__main__':
    data,cityname = loadData('city.txt')
    km = KMeans(n_clusters=4)
    label = km.fit_predict(data)
    expenses = np.sum(km.cluster_centers_,axis=1)
    CityCluster = [[],[],[],[]]
    for i in range(len(cityname)):
        CityCluster[label[i]].append(cityname[i])
    for i in range(len(CityCluster)):
        print("Expenses:$.2f"%expenses[i])
        print(CityCluster[i])

**应用2：DBSCAN密度聚类学生上网时间分布**
- 该算法将数据点分为三类：
  - 边界点：邻域内点不超过`minpts`个
  - 核心点：邻域内点超过了`minpts`个
- 算法流程：
  - 将所有点初始标记
  - 删除噪声点
  - 为距离在`eps`之内的所有核心点赋予一条边
  - 每组联通的核心点形成一个簇
  - 将每个边界点指派到一个与之关联的核心点的簇中。

In [ ]:
import numpy as np
import sklearn.cluster as skc
from sklearn import metrics
import matplotlib.pyplot as plt

mac2id=dict()
onlinetimes=[]
f=open('data.txt','r')
for line in f:
    mac = line.split(',')[2]
    onlinetime = int(line.split(',')[6])
    starttime = int(line.split(',')[4].split(' '[1].split(':')[0]))
    if mac not in mac2id:
        mac2id[mac] = len(onlinetimes)
        onlinetimes.append((starttime,onlinetime))
    else:
        onlinetimes[mac2id[mac]]=[(starttime,onlinetime)]
    real_X = np.array(onlinetimes.reshape((-1,2)))
    X = real_X[:,0:1]
    
    db=skc.DBSCAN(eps=0.01,min_samples=20).fit(X)
    labels = db.labels_
    
    print('Labels:')
    print(labels)
    ratio = len(labels[labels[:]==-1])/len(labels)
    print('Noise ratio: {%.2f}'%ratio)
    
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    
    print('Estimated number of clusters: %d' % n_clusters_)
    print('Silhouette Coefficient: %0.3f'\
        % metrics.silhouette_score(X, labels, metric='euclidean'))
    
    for i in range(n_clusters_):
        print('cluster ',i,':')
        print(list(X[labels==i])).flatten()
    
    plt.hist(X,24)